In [1]:
!pip -q install transformers datasets torch scikit-learn tqdm


In [2]:
import random
import numpy as np
import torch
import torch.nn as nn

from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import DataLoader

from transformers import BertTokenizer, BertModel
from sklearn.metrics import classification_report, precision_recall_fscore_support


In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [4]:
dataset = load_dataset("amazon_polarity")

train_dataset = dataset["train"].select(range(500))
test_dataset  = dataset["test"].select(range(5))  # tiny for quick runs per assignment


README.md: 0.00B [00:00, ?B/s]

C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shweiss\.cache\huggingface\hub\datasets--amazon_polarity. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


amazon_polarity/train-00000-of-00004.par(…):   0%|          | 0.00/260M [00:00<?, ?B/s]

amazon_polarity/train-00001-of-00004.par(…):   0%|          | 0.00/258M [00:00<?, ?B/s]

amazon_polarity/train-00002-of-00004.par(…):   0%|          | 0.00/255M [00:00<?, ?B/s]

amazon_polarity/train-00003-of-00004.par(…):   0%|          | 0.00/254M [00:00<?, ?B/s]

amazon_polarity/test-00000-of-00001.parq(…):   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

MAX_LEN = 128  # you can tune this later

def tokenize_function(example):
    # amazon_polarity has fields: "title", "content", "label"
    # We'll combine title + content for stronger signal.
    text = (example["title"] + " " + example["content"]).strip()

    return tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

train_dataset = train_dataset.map(tokenize_function)
test_dataset  = test_dataset.map(tokenize_function)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shweiss\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [6]:
columns = ["input_ids", "attention_mask", "token_type_ids", "label"]
train_dataset.set_format(type="torch", columns=columns)
test_dataset.set_format(type="torch", columns=columns)


In [7]:
BATCH_SIZE = 16  # tune later

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE)


In [8]:
class CustomBERTClassifier(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased", num_labels=2, dropout=0.3):
        super().__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        pooled_output = outputs.pooler_output  # [batch, hidden]
        x = self.dropout(pooled_output)
        logits = self.classifier(x)
        return logits


In [9]:
model = CustomBERTClassifier().to(device)

criterion = nn.CrossEntropyLoss()

# Simple Adam optimizer (fine for assignment). You can switch to AdamW if you want.
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [10]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(dataloader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()

        logits = model(input_ids, attention_mask, token_type_ids)
        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / max(total, 1)
    avg_acc = correct / max(total, 1)
    print(f"Train Loss: {avg_loss:.4f} | Train Acc: {avg_acc:.4f}")


In [11]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            labels = batch["label"].to(device)

            logits = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(logits, labels)

            total_loss += loss.item() * labels.size(0)
            preds = torch.argmax(logits, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

    avg_loss = total_loss / max(total, 1)
    avg_acc = correct / max(total, 1)

    print(f"Eval Loss: {avg_loss:.4f} | Eval Acc: {avg_acc:.4f}")
    return all_labels, all_preds


In [12]:
NUM_EPOCHS = 3

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
    train_one_epoch(model, train_loader, optimizer, criterion, device)
    labels, preds = evaluate(model, test_loader, criterion, device)



Epoch 1/3


Training: 100%|████████████████████████████████████████████████████████████████████████| 32/32 [03:46<00:00,  7.09s/it]


Train Loss: 0.6345 | Train Acc: 0.6400


Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.87it/s]


Eval Loss: 0.4368 | Eval Acc: 0.8000

Epoch 2/3


...

Eval Loss: 0.5501 | Eval Acc: 0.8000

Epoch 3/3


Training: 100%|████████████████████████████████████████████████████████████████████████| 32/32 [03:44<00:00,  7.03s/it]


Train Loss: 0.1987 | Train Acc: 0.9400


Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.55it/s]

Eval Loss: 0.3715 | Eval Acc: 0.8000


In [13]:
print("\nClassification Report:")
print(classification_report(labels, preds, digits=4))



Classification Report:
              precision    recall  f1-score   support

           0     0.5000    1.0000    0.6667         1
           1     1.0000    0.7500    0.8571         4

    accuracy                         0.8000         5
   macro avg     0.7500    0.8750    0.7619         5
weighted avg     0.9000    0.8000    0.8190         5



In [14]:
label_map = {0: "negative", 1: "positive"}

def predict_sentiment(text: str):
    model.eval()
    encoded = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
        return_tensors="pt"
    )
    input_ids = encoded["input_ids"].to(device)
    attention_mask = encoded["attention_mask"].to(device)
    token_type_ids = encoded.get("token_type_ids", None)
    if token_type_ids is not None:
        token_type_ids = token_type_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask, token_type_ids)
        pred = torch.argmax(logits, dim=1).item()

    return label_map[pred]

sample_text = "The product quality was amazing and it exceeded my expectations!"
print("Predicted sentiment:", predict_sentiment(sample_text))


Predicted sentiment: positive


In [15]:
class CustomBERTClassifierV2(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased", num_labels=2, dropout=0.3, hidden_dim=256):
        super().__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)

        self.dropout1 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, hidden_dim)
        self.act = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        x = outputs.pooler_output
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout2(x)
        logits = self.fc2(x)
        return logits


In [16]:
model_v2 = CustomBERTClassifierV2(hidden_dim=256, dropout=0.3).to(device)
criterion_v2 = nn.CrossEntropyLoss()
optimizer_v2 = torch.optim.Adam(model_v2.parameters(), lr=2e-5)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [17]:
def run_experiment(model_factory, train_ds, test_ds, *, batch_size, lr, epochs, name):
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    test_loader  = DataLoader(test_ds, batch_size=batch_size)

    model = model_factory().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        print(f"\n[{name}] Epoch {epoch+1}/{epochs}")
        train_one_epoch(model, train_loader, optimizer, criterion, device)
        labels, preds = evaluate(model, test_loader, criterion, device)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = sum(int(p == y) for p, y in zip(preds, labels)) / max(len(labels), 1)

    result = {
        "experiment": name,
        "batch_size": batch_size,
        "lr": lr,
        "epochs": epochs,
        "accuracy": acc,
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
        "report": classification_report(labels, preds, digits=4)
    }
    return result


In [18]:
def base_factory():
    return CustomBERTClassifier(dropout=0.3)

def v2_factory():
    return CustomBERTClassifierV2(hidden_dim=256, dropout=0.3)


In [19]:
results = []

results.append(
    run_experiment(
        v2_factory, train_dataset, test_dataset,
        batch_size=32, lr=2e-5, epochs=5,
        name="Combo1: V2 head (2-layer), epochs=5, bs=32, lr=2e-5"
    )
)

results.append(
    run_experiment(
        v2_factory, train_dataset, test_dataset,
        batch_size=16, lr=2e-5, epochs=5,
        name="Combo2: V2 head (2-layer), epochs=5, bs=16, lr=2e-5"
    )
)

for r in results:
    print("\n", r["experiment"])
    print(f"Acc={r['accuracy']:.4f} | P={r['precision']:.4f} | R={r['recall']:.4f} | F1={r['f1']:.4f}")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



[Combo1: V2 head (2-layer), epochs=5, bs=32, lr=2e-5] Epoch 1/5


Training: 100%|████████████████████████████████████████████████████████████████████████| 16/16 [03:16<00:00, 12.30s/it]


Train Loss: 0.6832 | Train Acc: 0.5400


...

Eval Loss: 0.1947 | Eval Acc: 1.0000

[Combo2: V2 head (2-layer), epochs=5, bs=16, lr=2e-5] Epoch 5/5


Training: 100%|████████████████████████████████████████████████████████████████████████| 32/32 [04:10<00:00,  7.84s/it]


Train Loss: 0.0904 | Train Acc: 0.9760


Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.70it/s]

Eval Loss: 0.1073 | Eval Acc: 1.0000

 Combo1: V2 head (2-layer), epochs=5, bs=32, lr=2e-5
Acc=0.8000 | P=1.0000 | R=0.7500 | F1=0.8571

 Combo2: V2 head (2-layer), epochs=5, bs=16, lr=2e-5
Acc=1.0000 | P=1.0000 | R=1.0000 | F1=1.0000


In [20]:
class CustomBERTClassifierV3(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased", num_labels=2, dropout=0.3, h1=256, h2=128):
        super().__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)

        self.net = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.bert.config.hidden_size, h1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h2, num_labels),
        )

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        x = outputs.pooler_output
        return self.net(x)

def v3_factory():
    return CustomBERTClassifierV3(dropout=0.3, h1=256, h2=128)


In [21]:
def evaluate_with_metrics(model, dataloader, criterion, device):
    labels, preds = evaluate(model, dataloader, criterion, device)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = sum(int(p == y) for p, y in zip(preds, labels)) / max(len(labels), 1)

    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(labels, preds, digits=4))

    return {"accuracy": acc, "precision": float(precision), "recall": float(recall), "f1": float(f1)}


In [ ]:
metrics = evaluate_with_metrics(model_v2, test_loader, criterion_v2, device)
metrics


Evaluating: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.44s/it]

Eval Loss: 0.7210 | Eval Acc: 0.2000
Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000

Classification Report:
              precision    recall  f1-score   support

           0     0.2000    1.0000    0.3333         1
           1     0.0000    0.0000    0.0000         4

    accuracy                         0.2000         5
   macro avg     0.1000    0.5000    0.1667         5
weighted avg     0.0400    0.2000    0.0667         5




C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", resu

{'accuracy': 0.2, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}

In [ ]:
torch.save(model_v2.state_dict(), "bert_amazonpolarity_v2.pt")
print("Saved!")


Saved!
